In [1]:
import os
import json

os.chdir('D:\\dev\\projects\\artifact-aware-assistant\\backend\\app\\routes\\api')
os.environ['no_proxy'] = "10.227.91.60"

# from conversation_openai import Conversation, DumbConversation, Artifact, Tool
# from example_tools import tools
from conversation import SYSTEM_MESSAGE, Artifact, Tool

from dotenv import load_dotenv

load_dotenv()  # Load environment variables

True

In [2]:
def get_listing(address):
    content = f"""\
{{
    "address": "{address}",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}}
"""

    artifact = Artifact(
        identifier="18bacG4a",
        type="application/json",
        title=address,
        content=content)
    return str(artifact)


def get_email_template():
    artifact = Artifact(
        identifier="98acb34d", 
        type="text/plain", 
        title="Prospective Buyer Listing Email Template", 
        content="""
Dear {buyer_name},

I wanted to bring to your attention an exciting new listing at {address}, {city}, {state} that I believe would be perfect for you.

This beautiful {beds} bedroom, {baths} bathroom home offers {sqft} square feet of living space and was built in {year_built}. It is currently listed at ${price:,}, which represents excellent value for this desirable neighborhood.

Some key features that make this property stand out:
- Spacious layout with {beds} bedrooms
- {sqft} square feet of living space
- Well-maintained home built in {year_built}
- Current Zestimate: ${zestimate:,}

To give you some context about the local market, there are several comparable properties in the immediate vicinity:
- A similar {beds} bed/{baths} bath home just {distance_miles} miles away recently sold for ${last_sold[price]:,}
- Nearby properties range from ${price:,} to ${price:,} in this area
- Most homes in this neighborhood were built in the 1970s-1980s

Would you like to schedule a viewing of this property? I have several time slots available this week and would be happy to show you around.

Best regards,
Your Real Estate Agent
""")
    return str(artifact)


get_listing_schema = {
    "name": "get_listing",
    "description": "Get details about a specific property listing",
    "parameters": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The street address to look up"
            }
        },
        "required": ["address"],
        "additionalProperties": False
    }
}


get_email_template_schema = {
    "name": "get_email_template",
    "description": "Get an email template for sending property listings to prospective buyers. This is just the template, not the actual email. This template should not be edited unless the user explicitly asks to edit it. The contents of the actual instantiated email, however, can be edited.",
    "parameters": {
        "type": "object",
        "properties": {},
        "additionalProperties": False
    }
}


tools = [
    Tool(get_listing_schema, get_listing),
    Tool(get_email_template_schema, get_email_template),
]

In [3]:
from openai.types.chat.chat_completion import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall, Function

def process_tool_uses_and_results(messages):
    '''
    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get information about the property at 192 Oak St.", 'type': 'text'}, 
        {'id': 'toolu_012m2VtyXxFL4RBGHzJqSN4R', 'input': {'address': '192 Oak St'}, 'name': 'get_listing', 'type': 'tool_use'}
        ]
    }
    
    {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_012m2VtyXxFL4RBGHzJqSN4R', 'content': '<a href="#18bacG4a">192 Oak St</a>'}]}
    
    ->

    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get information about the property at 192 Oak St.", 'type': 'text'}, 
        {'type': 'tool_use', 'name': 'get_listing', 'input': {'address': '192 Oak St'}, 'output': '<a href="#18bacG4a">192 Oak St</a>'}
        ]
    }
    
    '''
    processed_messages = []
    tool_uses_content = None
    
    for message in messages:
        if type(message) == ChatCompletionMessage:
            # Store tool uses for next iteration
            tool_uses_content = message
            num_tools = len(tool_uses_content.tool_calls)
            tool_idx = 0
            processed_content = []
            continue

        if message['role'] == 'tool' and tool_uses_content:
            # Create mapping of tool_use_id to result content
            tool_results_map = {message['tool_call_id']: message['content']}
            
            # Process the content list, replacing tool uses with combined use+result
            item = tool_uses_content.tool_calls[tool_idx]
            processed_content.append({
                'type': 'tool_use',
                'name': item.function.name,
                'input': json.loads(item.function.arguments),
                'output': tool_results_map[item.id]
                                 })
                    
            tool_idx += 1
            if tool_idx == num_tools:
                processed_messages.append({
                    'role': 'assistant',
                    'content': processed_content
                })
                tool_uses_content = None
            
            continue
            
        # Add any other messages as-is
        processed_messages.append(message)
        
    return processed_messages


def unprocess_tool_uses_and_results(messages):
    '''
    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get information about the property at 192 Oak St.", 'type': 'text'}, 
        {'type': 'tool_use', 'name': 'get_listing', 'input': {'address': '192 Oak St'}, 'output': '<a href="#18bacG4a">192 Oak St</a>'}
        ]
    }

    -> 

    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get the listing information for 192 Oak St.", 'type': 'text'}, 
        {'type': 'tool_use', 'id': 'toolu_0', 'name': 'get_listing', 'input': {'address': '192 Oak St'}}
        ]
    }

    {'role': 'user', 
    'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0', 'content': '<a href="#18bacG4a">192 Oak St</a>'}]}
    
    '''
    
    unprocessed_messages = []
    tool_use_counter = 0
    
    for message in messages:
        if message['role'] == 'assistant' and isinstance(message['content'], list):
            assistant_message_content = []
            user_message_content = []
            tool_calls = []
            for item in message['content']:
                if item.get('type') == 'tool_use':
                    # Generate unique tool use ID
                    tool_use_id = f'toolu_{tool_use_counter}'
                    tool_use_counter += 1
                    
                    # Split tool use and result into separate messages
                    tool_call = ChatCompletionMessageToolCall(
                        id=tool_use_id, 
                        function=Function(arguments=json.dumps(item['input']), name=item['name']), 
                        type='function'
                    )

                    tool_calls.append(tool_call)
                    
                    # Store tool result for later
                    user_message_content.append({
                        'role': 'tool',
                        'tool_call_id': tool_use_id,
                        'content': item['output'],
                    })
                else:
                    assistant_message_content.append(item)

            assistant_message_content.append(ChatCompletionMessage(role='assistant', tool_calls=tool_calls))
            
            unprocessed_messages.extend(assistant_message_content)
            
            unprocessed_messages.extend(user_message_content)
            
        else:
            # Add any other messages as-is
            unprocessed_messages.append(message)
    
    return unprocessed_messages

In [4]:
import openai
import re

class Conversation:
    def __init__(self, tools=None, messages=None, artifacts=None):
        self.client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.model = "gpt-4-turbo"
        self.messages = messages or []
        self.artifacts = artifacts or []
        self.tools = tools or []

    def say(self, message):
        system_message = self._generate_system_message(self.artifacts)
        tools = [{"type": "function", "function": t.schema} for t in self.tools]

        self.messages.insert(0, {
            "role": "system",
            "content": system_message
        })

        self.messages.append({
            "role": "user",
            "content": message
        })

        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            max_tokens=3000,
            temperature=0.7,
            tools=tools,
        )

        # Handle potential tool use
        while response.choices[0].finish_reason == "tool_calls":
            tool_result_messages = []
            for block in response.choices[0].message.tool_calls:
                if block.type == "function":
                    tool_use = block
                    tool_name = tool_use.function.name
                    tool_input = json.loads(tool_use.function.arguments)
                    tool_result = self._process_tool_call(tool_name, tool_input)
                    tool_result_messages.append({
                        "role": "tool",
                        "tool_call_id": tool_use.id,
                        "content": tool_result,
                    })

            self.messages.append(response.choices[0].message)
            self.messages += tool_result_messages

            # Get final response after tool use
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                max_tokens=3000,
                temperature=0.7,
                tools=tools,
            )

        assistant_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_message})
        del self.messages[0]
        artifacts, messages = self._extract_messages_and_artifacts()

        return {
            'messages': messages,
            'artifacts': artifacts,
        }

    def _process_tool_call(self, tool_name, tool_input):
        for tool in self.tools:
            if tool.name == tool_name:
                return tool.callable(**tool_input)
        raise Exception(f"Tool {tool_name} not found")

    def _generate_system_message(self, artifacts):
        artifacts_info = "\n".join([str(artifact) for artifact in artifacts])
        system_message = SYSTEM_MESSAGE

        if artifacts:
            system_message += f"""\
            <artifacts>
            {artifacts_info}
            </artifacts>
            """
        return system_message

    def _extract_messages_and_artifacts(self):
        """
        Extracts artifacts from the conversation and returns a list of messages with the artifacts replaced by anchor tags.
        """
        artifacts = self.artifacts
        new_messages = []

        # Process each message
        for message in self.messages:
            if not isinstance(message, dict):
                new_messages.append(message)
                continue

            content = message['content']
            new_message = {"role": message["role"]}

            if message["role"] == 'tool':
                new_message['tool_call_id'] = message['tool_call_id']

            # Process string content
            new_content, message_artifacts = self._process_content(content)
            artifacts.extend(message_artifacts)
            new_message["content"] = new_content
            new_messages.append(new_message)

        # Remove duplicate artifacts keeping only the latest version
        seen_ids = {}
        unique_artifacts = []
        for artifact in artifacts:
            seen_ids[artifact.identifier] = artifact
        unique_artifacts = list(seen_ids.values())

        return unique_artifacts, new_messages

    def _process_content(self, text):
        """Helper method to process text content and extract artifacts"""
        artifacts = []

        # Find all artifact blocks using regex
        artifact_pattern = r'<artifact\s+identifier="([^"]+)"\s+type="([^"]+)"\s+title="([^"]+)">(.*?)</artifact>'

        # Keep track of where we last ended to build the new content
        last_end = 0
        new_content = ""

        for match in re.finditer(artifact_pattern, text, re.DOTALL):
            # Add any text before this match
            new_content += text[last_end:match.start()]

            # Extract artifact info
            identifier = match.group(1)
            type_ = match.group(2)
            title = match.group(3)
            content = match.group(4).strip()

            # Create and store artifact
            artifact = Artifact(identifier, type_, title, content)
            artifacts.append(artifact)

            # Add anchor tag
            new_content += f'<a href="#{identifier}">{title}</a>'

            last_end = match.end()

        # Add any remaining text
        new_content += text[last_end:]

        return new_content, artifacts

In [5]:
messages = [
{'role': 'user', 'content': 'Hello!'},

{'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"},

{'role': 'user', 'content': 'I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing and email template?'},

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_up3YbSEJJf3VhAehVcojSfIS', function=Function(arguments='{"address": "192 Oak St"}', name='get_listing'), type='function'), ChatCompletionMessageToolCall(id='call_T2hc6lbLWmqM2kR7wkNUuRyt', function=Function(arguments='{}', name='get_email_template'), type='function')]),

{'role': 'tool', 'tool_call_id': 'call_up3YbSEJJf3VhAehVcojSfIS', 'content': '<a href="#18bacG4a">192 Oak St</a>'},

{'role': 'tool', 'tool_call_id': 'call_T2hc6lbLWmqM2kR7wkNUuRyt', 'content': '<a href="#98acb34d">Prospective Buyer Listing Email Template</a>'},

{'role': 'assistant', 'content': 'I have pulled up the listing details for the home at 192 Oak St and also retrieved an email template for sending property listings to prospective buyers.\n\nYou can view the listing details in the Artifact Viewer here: <a href="#18bacG4a">192 Oak St listing details</a>.\n\nThe email template is also available for review and customization here: <a href="#98acb34d">Prospective Buyer Listing Email Template</a>.\n\nPlease let me know if you would like to customize the email template with specific details from the listing or any additional information.'},

{'role': 'user', 'content':  'The buyer name is Abbas, can you write the email? Show the draft in the Artifact Viewer'}
]
user_message = messages[-1]['content']
messages = messages[: -1]
messages = unprocess_tool_uses_and_results(process_tool_uses_and_results(messages))
artifacts = [] #convert_to_artifacts()



In [6]:
conversation = Conversation(
    tools=tools,
    messages=messages,
    artifacts=artifacts,
)

response = conversation.say(user_message)

In [7]:
messages = response['messages']
artifacts = response['artifacts']

In [8]:
for m in messages:
    print(m)
    print()

{'role': 'user', 'content': 'Hello!'}

{'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}

{'role': 'user', 'content': 'I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing and email template?'}

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='toolu_0', function=Function(arguments='{"address": "192 Oak St"}', name='get_listing'), type='function'), ChatCompletionMessageToolCall(id='toolu_1', function=Function(arguments='{}', name='get_email_template'), type='function')])

{'role': 'tool', 'tool_call_id': 'toolu_0', 'content': '<a href="#18bacG4a">192 Oak St</a>'}

{'role': 'tool', 'tool_call_id': 'toolu_1', 'content': '<a href="#98acb34d">Prosp

In [9]:
artifacts

[Artifact(identifier="3baf9f83", title="Customized Email for Abbas")]

In [10]:
for m in process_tool_uses_and_results(messages):
    print(m)
    print()

{'role': 'user', 'content': 'Hello!'}

{'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}

{'role': 'user', 'content': 'I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing and email template?'}

{'role': 'assistant', 'content': [{'type': 'tool_use', 'name': 'get_listing', 'input': {'address': '192 Oak St'}, 'output': '<a href="#18bacG4a">192 Oak St</a>'}, {'type': 'tool_use', 'name': 'get_email_template', 'input': {}, 'output': '<a href="#98acb34d">Prospective Buyer Listing Email Template</a>'}]}

{'role': 'assistant', 'content': 'I have pulled up the listing details for the home at 192 Oak St and also retrieved an email template for sending property listings to prospective buyers.\n\nYou can view the listing details in the Artifact Viewe

In [11]:
for m in unprocess_tool_uses_and_results(process_tool_uses_and_results(messages)):
    print(m)
    print()

{'role': 'user', 'content': 'Hello!'}

{'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}

{'role': 'user', 'content': 'I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing and email template?'}

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='toolu_0', function=Function(arguments='{"address": "192 Oak St"}', name='get_listing'), type='function'), ChatCompletionMessageToolCall(id='toolu_1', function=Function(arguments='{}', name='get_email_template'), type='function')])

{'role': 'tool', 'tool_call_id': 'toolu_0', 'content': '<a href="#18bacG4a">192 Oak St</a>'}

{'role': 'tool', 'tool_call_id': 'toolu_1', 'content': '<a href="#98acb34d">Prosp

In [12]:
ChatCompletionMessageToolCall(id='call_Hba03dQeFozJVjafyJH0Md9B', function=Function(arguments='{"address":"192 Oak St"}', name='get_listing'), type='function')

ChatCompletionMessageToolCall(id='call_Hba03dQeFozJVjafyJH0Md9B', function=Function(arguments='{"address":"192 Oak St"}', name='get_listing'), type='function')

In [13]:
ChatCompletionMessage(role='assistant', tool_calls=[])

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[])

In [14]:
json.loads("{\"address\":\"192 Oak St\"}")

{'address': '192 Oak St'}